In [31]:
import tensorflow as tf
import numpy as np

In [32]:
EPOCHS = 20
Batch_Size = 128
Shuffle = 1000
LR = 1e-4

1. 데이터 불러오기

In [33]:
from tensorflow.keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


2. 입력 데이터 전처리

In [34]:
X_mean = np.mean(X_train, axis=(0,1,2))
X_std = np.std(X_train, axis=(0,1,2))

X_train = (X_train - X_mean) /X_std
X_test = (X_test-X_mean) / X_std


In [35]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(35000, 32, 32, 3) (35000, 1) (15000, 32, 32, 3) (15000, 1)


In [36]:
train_tf = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_tf = train_tf.shuffle(1000).batch(64)

val_tf = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(128)

CNN 모델 빌드

In [37]:
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2

def my_model():
  inputs = Input(shape=(32, 32, 3))

  x = Conv2D(32, (3,3), padding = 'same', activation = 'relu')(inputs)
  x = Conv2D(64, (3,3), padding = 'same', activation = 'relu')(x)

  x = MaxPool2D((2, 2))(x)
  x = Dropout(rate=0.5)(x)

  x = Conv2D(128, (3,3), padding='same', activation='relu')(x)
  x = Conv2D(256, (3,3), padding='valid', activation='relu')(x)

  x = MaxPool2D((2,2))(x)
  x = Flatten()(x)
  x = Dense(512, activation='relu')(x)
  x = Dropout(rate=0.5)(x)
  x = Dense(10, activation = 'softmax')(x)

  model = Model(inputs = inputs, outputs = x)

  return model

model = my_model()



# 4. LOSS Function & Optimizer

In [38]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

loss_object = SparseCategoricalCrossentropy()
optimizer = Adam()

In [39]:
from tensorflow.keras.metrics import Mean
from tensorflow.keras.metrics import SparseCategoricalAccuracy

train_loss = Mean(name='train_loss')
train_accuracy = SparseCategoricalAccuracy(name='train_accuracy')

val_loss = Mean(name='val_loss')
val_accuracy = SparseCategoricalAccuracy(name='val_accuracy')

Back Propagation

In [40]:
@tf.function

def train_step(images, lables):
  with tf.GradientTape() as tape:
    outputs = model(images, training = True)

    loss = loss_object(labels, outputs)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, outputs)

@tf.function
def val_step(images, labels):
  outputs = model(images, training=False)
  v_loss = loss_object(labels, outputs)

  val_loss(v_loss)
  val_accuracy(labels, outputs)

학습하기

In [46]:
for epoch in range(EPOCHS) :
    train_loss.reset_states()
    train_accuracy.reset_states()
    val_loss.reset_states()
    val_accuracy.reset_states()
    
    for images, labels in train_tf :
        train_step(images, labels)
        
    for val_images, val_labels in val_tf:
        val_step(val_images, val_labels)
    print('Epoch: {:.3d}, Train_loss : {:.3f}, Train_acc: {:.3f}, val_loss: {:.3f}, val_acc: {:.3%}'.format(
    epoch+1, train_loss.result(), train_accuracy.result(), val_loss.result(), val_accuracy.result()))

ValueError: ignored

In [48]:
for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  val_loss.reset_states()
  val_accuracy.reset_states()

  for images, labels in train_tf:
    train_step(images, labels)

  for val_images, val_labels in val_tf:
    val_step(val_images, val_labels)

  print('Epoch: {:03d}, Train_loss: {:.3f}, Train_acc: {:.3%}, Val_loss: {:.3f}, Val_acc: {:.3%}'.format( 
        epoch + 1,
        train_loss.result(),
        train_accuracy.result(),
        val_loss.result(),
        val_accuracy.result()))

Epoch: 001, Train_loss: 2.201, Train_acc: 15.626%, Val_loss: 2.441, Val_acc: 9.713%
Epoch: 002, Train_loss: 2.201, Train_acc: 15.500%, Val_loss: 2.427, Val_acc: 9.713%
Epoch: 003, Train_loss: 2.200, Train_acc: 15.603%, Val_loss: 2.435, Val_acc: 9.713%
Epoch: 004, Train_loss: 2.200, Train_acc: 15.597%, Val_loss: 2.436, Val_acc: 9.713%
Epoch: 005, Train_loss: 2.200, Train_acc: 15.609%, Val_loss: 2.436, Val_acc: 9.713%
Epoch: 006, Train_loss: 2.200, Train_acc: 15.609%, Val_loss: 2.435, Val_acc: 9.713%
Epoch: 007, Train_loss: 2.200, Train_acc: 15.609%, Val_loss: 2.437, Val_acc: 9.713%
Epoch: 008, Train_loss: 2.201, Train_acc: 15.571%, Val_loss: 2.438, Val_acc: 9.713%
Epoch: 009, Train_loss: 2.200, Train_acc: 15.609%, Val_loss: 2.436, Val_acc: 9.713%
Epoch: 010, Train_loss: 2.200, Train_acc: 15.609%, Val_loss: 2.437, Val_acc: 9.713%
Epoch: 011, Train_loss: 2.200, Train_acc: 15.609%, Val_loss: 2.436, Val_acc: 9.713%
Epoch: 012, Train_loss: 2.200, Train_acc: 15.609%, Val_loss: 2.437, Val_acc: